# **BirdCLEF 2025 Data Preprocessing Notebook**
This notebook demonstrates how we can transform audio data into mel-spectrogram data. This transformation is essential for training 2D Convolutional Neural Networks (CNNs) on audio data, as it converts the one-dimensional audio signals into two-dimensional image-like representations.
I run this public notebook in debug mode(only a few sample processing). You can find the fully preprocessed mel spectrogram training dataset here --> [BirdCLEF'25 | Mel Spectrograms](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-mel-spectrograms).

In [20]:
import os
import cv2
import math
import time
import librosa
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pickle
import random
import torch
import warnings
warnings.filterwarnings("ignore")

In [21]:
class Config:
 
    DEBUG_MODE = True
    
    OUTPUT_DIR = './working/'
    DATA_ROOT = './Data'
    FS = 32000
    
    SEED = 42

    # Mel spectrogram parameters
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000

    EXCLUDE_HUMAN_VOICE = True
    NOHUMAN_DURATION = 5.0

    OVERSAMPLE_THRESHOLD = 200
    
    TARGET_DURATION = 5.0
    TARGET_SHAPE = (256, 256)  
    
    N_MAX = 50 if DEBUG_MODE else None  

config = Config()
random.seed(config.SEED)

In [22]:
print(f"Debug mode: {'ON' if config.DEBUG_MODE else 'OFF'}")
print(f"Max samples to process: {config.N_MAX if config.N_MAX is not None else 'ALL'}")

print("Loading taxonomy data...")
taxonomy_df = pd.read_csv(f'{config.DATA_ROOT}/taxonomy.csv')
species_class_map = dict(zip(taxonomy_df['primary_label'], taxonomy_df['class_name']))

print("Load vocal data...")
with open("train_voice_data.pkl", "rb") as fr :
    voice_dict = pickle.load(fr)

print("Loading training metadata...")
train_df = pd.read_csv(f'{config.DATA_ROOT}/train.csv')

Debug mode: ON
Max samples to process: 50
Loading taxonomy data...
Load vocal data...
Loading training metadata...


In [23]:
label_list = sorted(train_df['primary_label'].unique())
label_id_list = list(range(len(label_list)))
label2id = dict(zip(label_list, label_id_list))
id2label = dict(zip(label_id_list, label_list))

print(f'Found {len(label_list)} unique species')
working_df = train_df[['primary_label', 'rating', 'filename']].copy()
working_df['target'] = working_df.primary_label.map(label2id)
working_df['filepath'] = config.DATA_ROOT + '/train_audio/' + working_df.filename
working_df['samplename'] = working_df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])
working_df['class'] = working_df.primary_label.map(lambda x: species_class_map.get(x, 'Unknown'))
total_samples = min(len(working_df), config.N_MAX or len(working_df))
print(f'Total samples to process: {total_samples} out of {len(working_df)} available')
print(f'Samples by class:')
print(working_df['class'].value_counts())

Found 206 unique species
Total samples to process: 50 out of 28564 available
Samples by class:
class
Aves        27648
Amphibia      583
Mammalia      178
Insecta       155
Name: count, dtype: int64


In [ ]:
# voice_file_dict = {key[40:]: value for key, value in voice_dict.items()} # remove /kaggle/input/birdclef-2025/train_audio/

voice_file_dict = {config.DATA_ROOT+key[27:]: value for key, value in voice_dict.items()} # remove /kaggle/input/birdclef-2025/train_audio/

#audio_data, _ = librosa.load(row.filepath, sr=config.FS)
nv_file_dict = {}
print(f'Found {len(voice_file_dict)} files')
for (dir, vlist) in voice_file_dict.items() :
    audio_file, _ = librosa.load(dir, sr=config.FS)
    lenaudio = len(audio_file)
    nvlist = []
    nvlist.append({'start' : 0, 'end' : vlist[0]['start']-1})
    for i in range(len(vlist)-1) :
        nvlist.append({'start' : vlist[i]['end']+1, 'end' : vlist[i+1]['start']-1})
    if len(vlist)==1 :
        nvlist.append({'start' : vlist[0]['end']+1, 'end' : lenaudio})
    else :
        nvlist.append({'start' : vlist[i+1]['end']+1, 'end' : lenaudio})
    # check for too short count
    
    for j in reversed(range(len(nvlist))) :
        start = nvlist[j]['start']
        end = nvlist[j]['end']
        if (start+config.FS * config.TARGET_DURATION) >= end :
            nvlist.pop(j)
            # remove too short ones
    nv_file_dict[dir[19:]] = nvlist

# nv_file_dict : same format but stores no-voice range
print("Finished creating no-voice list")
    

Found 3846 files


Exception ignored in: <function tqdm.__del__ at 0x7fa8a368ee60>
Traceback (most recent call last):
  File "/home/celenort/.local/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/home/celenort/.local/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7fa8a368ee60>
Traceback (most recent call last):
  File "/home/celenort/.local/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/home/celenort/.local/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7fa8a368ee60>
Traceback (most recent call last):
  File "/home/celenort/.local/lib/python3.10/site-pac

In [25]:
def audio2melspec(audio_data):
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,
        sr=config.FS,
        n_fft=config.N_FFT,
        hop_length=config.HOP_LENGTH,
        n_mels=config.N_MELS,
        fmin=config.FMIN,
        fmax=config.FMAX,
        power=2.0
    )

    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)
    mel_spec_power_norm = (mel_spec - mel_spec.min()) / (mel_spec.max() - mel_spec.min() + 1e-8)

    
    return (mel_spec_norm, mel_spec_db, mel_spec_norm, mel_spec_power_norm)

In [26]:
## Changelog : Remove cyclic padding, only add zero padding (equally on both sides)

print("Starting audio processing...")
print(f"{'DEBUG MODE - Processing only 50 samples' if config.DEBUG_MODE else 'FULL MODE - Processing all samples'}")
start_time = time.time()

all_bird_data = {}
errors = []
skipcount  = 0

for i, row in working_df.iterrows(): #tqdm(working_df.iterrows(), total=total_samples):
    if config.N_MAX is not None and i >= config.N_MAX:
        break
    
    try:
        audio_data, _ = librosa.load(row.filepath, sr=config.FS)

        target_samples = int(config.TARGET_DURATION * config.FS)

        start_frame = -1

        if row.filename in voice_file_dict :
            nvlist = nv_file_dict[row.filename]
            if len(nvlist) == 0 :
                # too short, also contains human voice
                skipcount = skipcount+1
                continue
            for i in range(len(nvlist)):
                nvlist[i]['end'] = nvlist[i]['end']-target_samples
                # narrowing range for start idx
            
            # let's choose among nvlist!
            lengths = []
            total = 0
            for r in nvlist:
                length = r['end']-r['start']+1
                lengths.append((r['start'], length))
                total += length
            start_idx = random.randint(0, total-1)
            for start, length in lengths :
                if start_idx < length :
                    start_frame = start + start_idx
                    break
                start_idx -= length

        if len(audio_data) < target_samples:
            left_pad = int((target_samples - len(audio_data))/2)
            right_pad = int(target_samples-len(audio_data)-left_pad)
            audio_data = np.pad(audio_data, (left_pad, right_pad), mode='constant')
        if start_frame == -1 : # didn't pass through nv
            start_frame = random.randint(0, len(audio_data)-target_samples)
        
        cropped_audio = audio_data[start_frame:start_frame+target_samples]

        mel_spec = audio2melspec(cropped_audio)[0]

        if mel_spec.shape != config.TARGET_SHAPE:
            mel_spec = cv2.resize(mel_spec, config.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)

        all_bird_data[row.samplename] = mel_spec.astype(np.float32)
        
    except Exception as e:
        print(f"Error processing {row.filepath}: {e}")
        errors.append((row.filepath, str(e)))

end_time = time.time()
print(f"Processing completed in {end_time - start_time:.2f} seconds")
print(f"Successfully processed {len(all_bird_data)} files out of {total_samples} total")
print(f'Skipped processing too short {skipcount} files')
print(f"Failed to process {len(errors)} files")

Starting audio processing...
DEBUG MODE - Processing only 50 samples
Processing completed in 10.32 seconds
Successfully processed 50 files out of 50 total
Skipped processing too short 0 files
Failed to process 0 files


In [ ]:
np.save('data.npy', all_bird_data)

In [27]:
import matplotlib.pyplot as plt

samples = []
displayed_classes = set()

max_samples = min(4, len(all_bird_data))

for i, row in working_df.iterrows():
    if i >= (config.N_MAX or len(working_df)):
        break
        
    if row['samplename'] in all_bird_data:
        if config.DEBUG_MODE:
            if row['class'] not in displayed_classes:
                samples.append((row['samplename'], row['class'], row['primary_label']))
                displayed_classes.add(row['class'])
        else:
            if row['class'] not in displayed_classes:
                samples.append((row['samplename'], row['class'], row['primary_label']))
                displayed_classes.add(row['class'])
        
        if len(samples) >= max_samples:  
            break

if samples:
    plt.figure(figsize=(16, 12))
    
    for i, (samplename, class_name, species) in enumerate(samples):
        plt.subplot(2, 2, i+1)
        plt.imshow(all_bird_data[samplename], aspect='auto', origin='lower', cmap='viridis')
        plt.title(f"{class_name}: {species}")
        plt.colorbar(format='%+2.0f dB')
    
    plt.tight_layout()
    debug_note = "debug_" if config.DEBUG_MODE else ""
    plt.savefig(f'{debug_note}melspec_examples.png')
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
import numpy as np
data = np.load('Test1.npy', allow_pickle=True)

In [3]:
print(data)

{'1139490-CSA36385': array([[0.6501087 , 0.81016314, 0.8441024 , ..., 0.5008096 , 0.7289406 ,
        0.92846   ],
       [0.6556635 , 0.8165829 , 0.8529962 , ..., 0.4972914 , 0.73530287,
        0.9380286 ],
       [0.666773  , 0.82942253, 0.8707838 , ..., 0.4902549 , 0.74802744,
        0.95716584],
       ...,
       [0.2326687 , 0.26610819, 0.27345017, ..., 0.09432434, 0.2623796 ,
        0.34882104],
       [0.23767272, 0.2700556 , 0.26857   , ..., 0.08699758, 0.26180002,
        0.350706  ],
       [0.24017474, 0.27202928, 0.26612994, ..., 0.08333419, 0.26151022,
        0.35164848]], shape=(256, 256), dtype=float32), '1139490-CSA36389': array([[0.84162146, 0.83914673, 0.84579724, ..., 0.7219672 , 0.74369335,
        0.70541686],
       [0.85514486, 0.8631656 , 0.8676349 , ..., 0.72079587, 0.74315906,
        0.70514166],
       [0.88219154, 0.9112034 , 0.91131014, ..., 0.7184532 , 0.74209034,
        0.7045913 ],
       ...,
       [0.2481362 , 0.24808656, 0.24727312, ..., 0.227